<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reading-data" data-toc-modified-id="Reading-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reading data</a></span></li><li><span><a href="#Prepartion-of-the-spectra" data-toc-modified-id="Prepartion-of-the-spectra-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepartion of the spectra</a></span></li><li><span><a href="#Host-galaxy-subtraction" data-toc-modified-id="Host-galaxy-subtraction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Host-galaxy subtraction</a></span></li><li><span><a href="#Creation-of-the-predefined-lists-of-emission-lines" data-toc-modified-id="Creation-of-the-predefined-lists-of-emission-lines-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creation of the predefined lists of emission lines</a></span></li><li><span><a href="#Defining-the-fitting-model" data-toc-modified-id="Defining-the-fitting-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Defining the fitting model</a></span></li><li><span><a href="#Fittings" data-toc-modified-id="Fittings-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fittings</a></span></li><li><span><a href="#Plotting-the-results" data-toc-modified-id="Plotting-the-results-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Plotting the results</a></span></li><li><span><a href="#Inspecting-&amp;-saving-the-plotting-results" data-toc-modified-id="Inspecting-&amp;-saving-the-plotting-results-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Inspecting &amp; saving the plotting results</a></span></li><li><span><a href="#Simple-analysis" data-toc-modified-id="Simple-analysis-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Simple analysis</a></span></li></ul></div>

**Nicolás Guerra-Varas**

Prof. Dragana Ilić

Tutor Isidora Jankov

# FANTASY Tutorial

The goal of this tutorial is to learn how to use __[`fantasy`](https://fantasy-agn.readthedocs.io/en/latest/index.html)__ to model AGN spectra.

## Imports

In [ ]:
%matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

import numpy as np
import pandas as pd

import glob

In [ ]:
# additional imports
import math as m
import csv
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table

In [ ]:
from fantasy_agn.tools import read_sdss, read_text, read_gama_fits

from fantasy_agn.models import (create_input_folder, create_feii_model, create_model, 
                               create_tied_model, continuum, create_line, create_fixed_model)

# Replicate Procedure

## Read spectrum

In [ ]:
# assigned spectrum
s = read_sdss('spec-0572-52289-0442.fits')
s_or = read_sdss('spec-0572-52289-0442.fits')
# saving original spectrum to see how it changes through the procedure

# these are the parameters that the spectrum has
# s.z redshift
# s.ra
# s.dec
# s.mjd
# s.plate
# s.fiber
# s.name
# s.err
# s.flux
# s.wave
# s.fwhm
# s.velscale

First I took a quick look into the spectrum.

In [ ]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12,5))

plt.plot(s.wave, s.flux, color='navy', label='Obs', lw=1)

plt.legend(loc='upper right', prop={'size': 15}, frameon=False, ncol=2)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_original.pdf')

plt.show()

I looked it up on SDSS/SIMBAD to get more information on this object.

In [ ]:
print(s.plate, s.mjd, s.fiber)

Here's an image of the object from the __[SDSS viewer](https://skyserver.sdss.org/dr18/VisualTools/explore/summary?plate=0572&mjd=52289&fiber=442)__

<img src='sdss_viewer.jpg' width="400" height="400">

Then I wanted to see where in the sky this is.

In [ ]:
ra = s.ra
dec = s.dec
eq = SkyCoord(ra, dec, frame='galactic', unit=u.deg)
gal = eq.galactic

plt.style.use('ggplot')

fig = plt.figure(figsize=(6,5))
plt.subplot(111, projection='aitoff')

plt.plot(gal.l.wrap_at(180*u.deg), gal.b.wrap_at(180*u.deg), linestyle='None')
plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian, marker='*', color='navy', s=150)

plt.grid(True, alpha=0.5)

plt.tick_params(axis='both', which='major', labelsize=5)

# plt.savefig('map_qso.pdf')

plt.show()

It is along the galactic plane. Maybe it will have significant Galactic absorption.

In [ ]:
print('Redshift =', s.z)

Finally, I checked its redshift, which indicates it is a distant object.

## Prepare spectra

In [ ]:
s.DeRedden()  # correct for Galactic extinction
s.CorRed()  # correct for cosmological redshift

I corrected for redshift and extinction and plotted against the unchanged data to see how it changes.

In [ ]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12,5))

plt.plot(s_or.wave, s_or.flux, '--', color='steelblue', label='Obs Original', lw=1)
plt.plot(s.wave, s.flux, color='navy', label='Obs Corrected', lw=1)

plt.legend(loc='upper right', prop={'size': 15}, frameon=False, ncol=1)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_pre_post_corrections.pdf')

plt.show()

## Host-galaxy subtraction

In [ ]:
# s.restore()
s.fit_host_sdss()  # fits for the host galaxy and AGN contibutions separately

s.host_no_mask = s.host  # saving the host

plt.legend(fontsize=12)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')
plt.title('Unmasked Host')

# plt.savefig('fit_host_sdss_default.pdf')

plt.show()

Here, the spectrum in red corresponds to the data. The blue and purple spectra are the fits to the host galaxy and the AGN respectively. Their summed spectrum gives the fit to the observations, which is quite good. The contribution of the host galaxy in this case is not negligible.

In [ ]:
s.restore()
s.fit_host_sdss(mask_host=True, custom=False)

s.host_masked = s.host  # saving the host

plt.legend(fontsize=12)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')
plt.title('Masked Host')

# plt.savefig('fit_host_sdss_masked.pdf')

plt.show()

I tried the host subtraction again, but now masking the main emission lines from the host galaxy ([O III], [N II], [S II]). The code does this by interpolating between the two edges of the lines instead of going through the line itself. The result was very different (see above). Now, the contribution of the host seems much less significant. The fit overall seems very good, just like the first try.

In [ ]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12,5))

plt.plot(s.wave, s.flux, color='navy', label='Obs Corrected', lw=1)

plt.legend(loc='upper left', prop={'size': 15}, frameon=False, ncol=1)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_post_host_sub.pdf')

plt.show()

In [ ]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12, 5))

plt.plot(s.wave, s.flux+s.host, color='navy', label='Obs' , lw=1.5)
plt.plot(s.wave, s.host_no_mask, color="#F10C45", label='Host: not masked', lw=1.0)
plt.plot(s.wave, s.host, color="green", label='Host: masked', lw=1.0)

plt.legend(loc='upper left', prop={'size': 10}, frameon=False)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_host_masked_vs_not.pdf')

plt.show()

In this plot, it is clear that the difference between both host fits are the lines.

## Creation of the predefined lists of emission lines

In [ ]:
# this is the minimum wavelength found in the spectrum
np.min(s.wave)

In [ ]:
# the information of the lines will be saved in the 'lines' folder
create_input_folder(xmin=np.min(s.wave), xmax=np.max(s.wave), path_to_folder='lines/', overwrite=True)

In this folder, there's a file for each of these lines, commonly found in AGN spectra: Balmer series [FeII], He, narrow lines, broad lines, [OIII], and others such as a broad and narrow line model.

In [ ]:
# cropping to a certain wavelength range

# I won't crop it initially
# s.crop(np.min(s.wave), np.max(s.wave))

# I'm cropping it for the 2nd attempt on
s.crop(3575, np.max(s.wave))

### Wavelengths of the lines

In [ ]:
H_lines = pd.read_csv('lines/hydrogen.csv')
O_N_lines = pd.read_csv('lines/oiii_nii.csv')
narrow_lines = pd.read_csv('lines/narrow_basic.csv')
narrowP = pd.read_csv('lines/narrow_plus.csv')

In [ ]:
H_alpha = np.float64(H_lines.loc[H_lines['line']=='Ha']['position'])
H_beta = np.float64(H_lines.loc[H_lines['line']=='Hb']['position'])

OIII_1 = np.float64(O_N_lines.loc[O_N_lines['line']=='[OIII]']['position'][0])
OIII_2 = np.float64(O_N_lines.loc[O_N_lines['line']=='[OIII]']['position'][1])
OIII_narrow = np.float64(narrow_lines.loc[narrow_lines['line']=='[OIII]']['position'])

NII_1 = np.float64(O_N_lines.loc[O_N_lines['line']=='[NII]']['position'][2])
NII_2 = np.float64(O_N_lines.loc[O_N_lines['line']=='[NII]']['position'][3])

SII_narrow_1 = np.float64(narrow_lines.loc[narrow_lines['line']=='[SII]']['position'][6])
SII_narrow_2 = np.float64(narrow_lines.loc[narrow_lines['line']=='[SII]']['position'][7])
SII_narrowP_1 = np.float64(narrowP.loc[narrowP['line']=='[SII]1F']['position'][7])
SII_narrowP_2 = np.float64(narrowP.loc[narrowP['line']=='[SII]1F']['position'][8])

## Defining the fitting model

            index    -0.7   dimensionless constant that defines the slope of the power law above 5000 Angstroms; it is equal to -0.7 in our fit to UVES spectra from Vestergaard & Peterson 1991 ApJ 374, 344; this value was changed from -0.5 to account for higher resolution data from Keck Observatories since that paper was published (see Section 2 of Smith et al 2006 ApJS 164, 508).  This parameter can be set between 0 and 3 inclusive with a limit placed on its value by requiring that it must be greater than or equal to min_index when evaluated at max_refer and less than or equal to max_index when evaluated at min_refer; see below for more details on these parameters which control

In [ ]:
# continumm
cont_1 = continuum(s, min_refer=5690, refer=5700, max_refer=5710)
cont_2 = continuum(s, 
                   # min_refer=4040, refer=4050, max_refer=4060, 
                   min_refer=4190, refer=4200, max_refer=4210, # best
                   # index1=-1.0, 
                   # index2=-1.5, min_index2=-2.5, max_index2=-0.5)
                   # index2=-2, min_index2=-3, max_index2=-1)
                   index2=-2.5, min_index2=-3.5, max_index2=-1.5)  # best
                   # index2=-3, min_index2=-4, max_index2=-2)

# default parameters:
# refer=5500, min_refer=5400, max_refer=7000, 
# index1=-1.7, min_index1=-3, max_index1=0, 
# index2=0, min_index2=-1, max_index2=1

# index1: sets the power law of the continuum
# index2: makes the continuum flat => I modified it

In [ ]:
# broad line component
broad_1 = create_fixed_model(['hydrogen.csv'], name='Broad')

In [ ]:
# narrow line component
narrow_1 = create_tied_model(name='OIII5007', 
                           files=['narrow_basic.csv', 'hydrogen.csv'], 
                           prefix='nr', 
                           fwhm=1000, 
                           min_offset=0, 
                           max_offset=300, 
                           min_fwhm=900, 
                           max_fwhm=1200, 
                           fix_oiii_ratio=True, 
                           position=5006.803341, 
                           included=True, 
                           min_amplitude=0.2)

In [ ]:
# helium
he_1 = create_fixed_model(['helium.csv'], name='He', fwhm=3000, min_fwhm=1000, max_fwhm=5000)

# He.amp_HeI_4144 is at its minimum boundary 0.0
# He.amp_HeI_4471 is at its minimum boundary 0.0
# He.fwhm is at its maximum boundary 5000.0

In [ ]:
# iron
fe_1 = create_feii_model(name='feii', 
                       fwhm=1800, 
                       min_fwhm=1000, 
                       max_fwhm=2000, 
                       offset=0,
                       min_offset=-3000, 
                       max_offset=3000)

'''fe_2 = create_feii_model(name='feii', 
                       fwhm=2200, 
                       min_fwhm=1000, 
                       max_fwhm=2500, 
                       offset=0,
                       min_offset=-3000, 
                       max_offset=3000)'''

In [ ]:
# Code fits simultaneously all features.
model_1st = cont_1 + broad_1 + narrow_1 + fe_1 + he_1
model_2nd = cont_2 + broad_1 + narrow_1 + fe_1 + he_1

So, I am trying different attempts:
1. No modifications
2. Fixing the continuum
3. Fixing the iron lines

## Fitting

In [ ]:
# 1st attempt
# s.fit(model_1st, ntrial=2)

# 2nd attempt
s.fit(model_2nd, ntrial=2)

In [ ]:
# it converged
print(s.gres.succeeded)

In [ ]:
# saving info on each fit attempt
attempts = {'chi2 1st': 124.23362044251917, 'iters 1st': 2, 
            'chi2 2nd': 5.209570577933846 # 0.1569414121840964
            , 'iters 2nd': 2
            }

# 5.209570577933846 with index2 = -2.5 & refer 4200

##  Inspecting & saving the plotting results

__[`sherpa`](https://sherpa.readthedocs.io/en/4.15.0/)__ saves the model's output into `gres`

In [ ]:
arr = np.stack([np.asarray(s.gres.parnames), np.asarray(s.gres.parvals)], axis=-1)
columns = ['parnames', 'parvalues']

fit_result = pd.DataFrame(arr, columns=columns)
fit_result

In [ ]:
# model_1st
model_2nd

In [ ]:
# To see the fitting results, one can use standard outputs from Sherpa package, such as:
# gres - to list all fitting results
# gres.parnames - to list all parameters
# save_json() - to save the results

print(s.gres.format())
s.save_json() #saving parameters 

In the first try, I didn't modify the parameters of the model. I got some warnings:
```
WARNING: parameter value feii.fwhm is at its maximum boundary 2000.0
WARNING: parameter value He.amp_HeI_4144 is at its minimum boundary 0.0
WARNING: parameter value He.amp_HeI_4471 is at its minimum boundary 0.0
WARNING: parameter value He.fwhm is at its maximum boundary 5000.0
```

I tried tuned the parameters of the model until they were resolved. Warning of the same type also came up during the second attempt.

---

I plotted the spectra with spectral lines indicated to get a better understanding of how well the model is fitting the data.

## Plotting the results

In [ ]:
'''plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(18,8))

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_1st(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_1st(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_1(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_1(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)
plt.title('Fantasy Fit: 1st Attempt (Iters: ' + str(attempts['iters 1st']) + r', $ \chi ^2 $ = ' + str(np.round(attempts['chi2 1st'], 3)) + ')')

plt.legend(loc='upper left',  prop={'size': 12}, frameon=False, ncol=2)

# plt.savefig('1st_attempt_fantasy_fit.pdf')

plt.show()'''

In [ ]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(18,8))

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_2nd(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_2nd(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_2(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_1(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)
plt.title('Fantasy Fit: 2nd Attempt (Iters: ' + str(attempts['iters 2nd']) + r', $ \chi ^2 $ = ' + str(np.round(attempts['chi2 2nd'], 3)) + ')')

plt.legend(loc='upper left',  prop={'size': 12}, frameon=False, ncol=2)

plt.savefig('2nd_attempt_fantasy_fit.pdf')

plt.show()

Next, I created an interactive plot so that I can zoom in and see how the lines are fitting in more detail.

In [ ]:
wavelengths = dict({SII_narrowP_1:'[SII] Narrow+, 1', 
                    SII_narrowP_2:'[SII] Narrow+, 2', 
                    OIII_narrow:'[OIII] Narrow', 
                    OIII_1:'[OIII], 1', 
                    OIII_2:'[OIII], 2', 
                    SII_narrow_1:'[SII] Narrow, 1', 
                    SII_narrow_2:'[SII] Narrow, 2',
                    NII_1:'[NII], 1', 
                    NII_2:'[NII], 2', 
                    H_alpha:'H alpha', 
                    H_beta:'H beta'})

colors = ['darkgreen', 'crimson', 'teal', 'goldenrod', 'slateblue', 'darkviolet', 'darkorange', 'royalblue', 
          'magenta', 'darkolivegreen', 'brown']

In [ ]:
plt.style.context(['nature', 'notebook'])
fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot()
fig.subplots_adjust(top=0.85)

for i in range(len(wavelengths)):
    ax.text(list(wavelengths.keys())[i], -200, list(wavelengths.values())[i], fontsize=10, rotation='vertical')
    ax.vlines(list(wavelengths.keys())[i], -100, 350, linestyles='dashed', label=list(wavelengths.values())[i], colors=colors[i])

plt.ion()

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_1st(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_1st(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_1(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_1(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.ylim(-220, 350)
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)

plt.title('Fantasy Fit: 1st Attempt (Iters: ' + str(attempts['iters 1st']) + r', $ \chi ^2 $ = ' + str(np.round(attempts['chi2 1st'], 3)) + ')')

plt.legend(bbox_to_anchor=(1.0, 0.8), prop={'size': 10}, frameon=False, ncol=1)

plt.show()

After the first attempt, the lines that need better fitting are:
* Continuum: below 4500 Ang, the continuum isn't steep enough
* [OIII] 1 and 2: the iron model doesn't have intense enough lines

### First Attempt
The first time I ran the model just the way it was to have a starting point. I believe the most important thing to fix is the continuum. The $\chi^2$ was equal to 

### Second Attempt
I focused on fixing the continuum. I changed the breakpoint of the power law (`refer`) and I made the left side of it steeper with the (`index2`).

# Measuring Lines

Narrow Ha, Hb, [OIII], [NII] and [SII] lines if present; Broad Ha, Hb.

In [ ]:
'''cont(s.wave)
narrow(s.wave)
broad(s.wave)
he(s.wave)
fe(s.wave)'''

In [ ]:
# measuring iron
flux_feII = np.sum(fe(s.wave))
flux_Ha_broad = np.sum
print(flux_feII)

# Calculating Line Ratios

## [NII]6583 / H $\alpha$

## [OIII]5007 / H $\beta$

## H $\alpha$ / H $\beta$

**References**

1. [Rakic, N., 2022, MNRAS, 516, 1624](https://ui.adsabs.harvard.edu/abs/2022MNRAS.516.1624R/abstract)
2. [Ilic, D. et al. 2020, A&A, 638, 13](https://ui.adsabs.harvard.edu/abs/2020A%26A...638A..13I/abstract)
3. [Kovacevic, J. et al. 2010, ApJS, 189, 15](https://ui.adsabs.harvard.edu/abs/2010ApJS..189...15K/abstract)
4. [Shapovalova, A. I. et al. 2012, ApJS, 202, 10](https://ui.adsabs.harvard.edu/abs/2012ApJS..202...10S/abstract)
5. Ilic, D., Rakic, N., Popovic, L. C., 2023, submitted